<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/PuLP%20-%20Tutorial/Tutorial%20PuLP%20(solution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial PuLP

This tutorial uses the well-known Traveling-Salesperson-Problem (TSP) to demonstrate the potential of Python's PuLP module. A thorough tutorial on the implementaion of the TSP in CPLEX can be found here: [TSP Tutorial on CPLEX](https://www.scm.bwl.uni-kiel.de/de/lehre/tutorial-on-cplex.pdf). There, the TSP and it's formulation is explained in more detail. 



## Problem description

The TSP is one of the most famous combinatorial problems in the fields of mathematics, computer science, and operations research. The classical definition of the TSP is as follows: What is the shortest possible route for a traveling salesperson seeking to visit each city on a list exactly once and return to his city of origin? (Cook, William (2012): *In Pursuit of the Travelin Salesman*).

While the problem is easy to understand and easy to formulate as a mathematical model, solving it to optimality through complete enumeration of all feasible solutions quickly becomes intractable as the number of potential solutions grows exponentially in the number of cities to visit.

To date, there exists no algorithm that solves the TSP to optimality in polynomial time and it is widely believed that there is no such algorithm. However, heuristics are usually capable of finding very good solutions in a short computation time and small problems can be solved to optimality using solvers. The latter is this tutorial's objective. 

## Mathematical formulation


This tutorial presents two graph-based formulations for the TSP. Both formulations yield to the same solution and use the following general notation:

**General notation:**
Let $N$ denote the number of nodes in the network, i.e., the number of cities. The distance for the salesperson to travel between any two nodes $i$ and $j$ is denoted with $c_{ij}$. Decision variable $x_{ij}$ is used to describe if the salesperson traverses from node $i$ to node $j$ ($x_{ij}=1$), or not ($x_{ij}=0$).

A general introduction on mathematical models for the TSP can be found in: Langevin, André, Francois Soumis, and Jacques Desrosiers. 1990. Classification of travelling salesman problem formulations. *Operations Research Letters*.

### Miller, Tucker and Zemlin (1960)

In the following, a formulation from Miller, Tucker, and Zemlin (1960), henceforth referred to as MTZ, is used. In addition to the general notation, the MTZ formulation requires one additional decision variable $u_i$ that describes the position of a node in the salesperson’s tour. Using this notation, the mathematical model for the TSP is as follows: 
<br><br>
$
\begin{align}
  \begin{array}{lll}
    &\textbf{Objective} & \\
    & \min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij} &~~~ (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & \sum\limits_{i=1}^{N} x_{ij} = 1,~ \forall~ j = 1,\dots,N  &~~~ (2) \\
    & \sum\limits_{j=1}^{N} x_{ij} = 1,~ \forall~ i = 1,\dots,N  &~~~ (3) \\
    & x_{ii} = 0,~ \forall~ i = 1,\dots,N  &~~~ (4) \\
    & u_{i} - u_{j} + N \cdot x_{ij} \le N -1,~ \forall~ i,j = 1,\dots,N: j \ne 1 ~\text{and}~ i \ne j  &~~~ (5) \\
    & u_{i} \in \mathbb{Z}^{\text{nonneg}},~ \forall~ i = 1,\dots,N  &~~~ (6) \\
    & x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N  &~~~ (7) \\
  \end{array}
\end{align}
$
<br><br>
Equation (1) states the objective (distance minimization) by summing up the distances of the selected arcs. Equations (2) to (4) ensure that the salesperson enters and exists each node exactly once (i.e., that each node is visited exactly once), respectively. Equations (5) restricts solutions with subtour using the MTZ formulation. In particular, this is done by stating that each customer is assigned to exactly one position, that each position is assigned to exactly one customer, and that the positions correspond to the actual tour of the salesperson (represented by $x_{ij}$). Finally, Equations (6) and (7) state the domains of the decision variables.

### Gavish and Graves (1976)

In the following, a formulation from Gavish and Graves (1976), henceforth referred to as GG, is used. It has a similar level of difficulty compared to the MTZ formulation. In particular, GG propose to use, in addition to the general notation, a two-dimensional integer decision variable $z_{ij}$ to track a travelled arc’s position in the salesperson’s tour. A very similar logic to the MTZ formulation, in which decision variable $u_i$ is used to track a city’s position in the salesperson’s tour. Using this notation, the mathematical model for the TSP is as follows: 
<br><br>
$
\begin{align}
  \begin{array}{lll}
    &\textbf{Objective} & \\
    & \min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij} &~~~ (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & \sum\limits_{i=1}^{N} x_{ij} = 1,~ \forall~ j = 1,\dots,N  &~~~ (2) \\
    & \sum\limits_{j=1}^{N} x_{ij} = 1,~ \forall~ i = 1,\dots,N  &~~~ (3) \\
    & x_{ii} = 0,~ \forall~ i = 1,\dots,N  &~~~ (4) \\
    &  \sum\limits_{j=1}^{N} z_{ij} - \sum\limits_{j=2}^{N} z_{ji} = 1,~ \forall~ i = 2,\dots,N &~~~ (5) \\
    & z_{ij} \le x_{ij} \cdot (N - 1),~ \forall~ i = 2,\dots,N, j = 1,\dots,N &~~~ (6) \\
    & z_{ij} \in \mathbb{Z}^{\text{nonneg}},~ \forall~ i,j = 1,\dots,N  &~~~ (7) \\
    & x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N  &~~~ (8) \\
  \end{array}
\end{align}
$
<br><br>
Equation (1) states the objective (distance minimization) by summing up the distances of the selected arcs. Equations (2) to (4) ensure that the salesperson enters and exists each node exactly once (i.e., that each node is visited exactly once), respectively. Equations (5) and (6) restrict solutions with subtour using the GG formulation. In particular, Equation (5) states that the value of the $z_{ij}$-variable is set so that the arc leaving a city is exactly one index position higher than the arc entering a city. The sum is used here as Equations (2) and (3) ensure that exactly one arc is entering and leaving each city, respectively. With this, the formulation forces the model to consider a single full cycle involving all the cities. Equation (6) ensures that the $z_{ij}$-variable can only take value 1 if the corresponding arc is also used in the tour (e.g., if $x_{ij}=1$). Finally, Equations (7) and (8) state the domains of the decision variables.

## Case study

We use an artificial small-sized dataset throughout this tutorial. The example consists of nine locations in which the first location ('Bahnhof')  is the salesperson's city of origin (also referred to as depot). The following table shows the latitude and longitude information of the locations:
<br><br>
\begin{array}{lll}
         name  & latitude & longitude \\
         \hline
      \text{Bahnhof} & 54.315487 & 10.132285 \\
\text{Friedrichsort} & 54.393713 & 10.184142 \\
     \text{Holtenau} & 54.374804 & 10.148470 \\
   \text{University} & 54.348125 & 10.117918 \\
        \text{Mitte} & 54.324606 & 10.136630 \\
       \text{Garden} & 54.313102 & 10.150910 \\
  \text{Wellingdorf} & 54.328293 & 10.179582 \\
   \text{Heikendorf} & 54.379798 & 10.212037 \\
        \text{Laboe} & 54.409884 & 10.232679 \\
\end{array}

## Python implementation

### Prepare input data

In [1]:
# load the required modules
!pip install pandas
!pip install plotly.express

import pandas as pd
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Prepare dataframe

In [2]:
# input data as matrix
input_data = [
     ['Bahnhof', 54.31548738087378, 10.132285213361302],
     ['Friedrichsort', 54.39371269762963, 10.184142058855787], 
     ['Holtenau', 54.37480425366618, 10.148469747635753], 
     ['University', 54.348125164196105, 10.117918463246932],
     ['Mitte', 54.32460566307153, 10.136629552088058],
     ['Garden', 54.313101710271425, 10.15091017118389],
     ['Wellingdorf', 54.32829259013629, 10.179582183151096],
     ['Heikendorf', 54.37979792093858, 10.212036804008633],
     ['Laboe', 54.409884128644144, 10.232678775594744]
]


# create dataframe from input data
df = pd.DataFrame(input_data, columns=['name', 'latitude', 'longitude'])

# ensure that longitude and latitude columns are numeric
df['longitude'] = pd.to_numeric(df['longitude'])
df['latitude'] = pd.to_numeric(df['latitude'])

# print dataframe 
print(df)

# illustrate input data via Plotly Express
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='name', 
                        zoom=10, mapbox_style='open-street-map',
                        color_discrete_map={'Bahnhof': '#000000'},
                        width=800, height=800)
fig.update_traces(marker={'size': 15})
fig.show()


            name   latitude  longitude
0        Bahnhof  54.315487  10.132285
1  Friedrichsort  54.393713  10.184142
2       Holtenau  54.374804  10.148470
3     University  54.348125  10.117918
4          Mitte  54.324606  10.136630
5         Garden  54.313102  10.150910
6    Wellingdorf  54.328293  10.179582
7     Heikendorf  54.379798  10.212037
8          Laboe  54.409884  10.232679


#### Get data from dataframe

In [4]:
!pip install haversine  # to measure the distance between two locations considering the Earth's surface
import haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
nodes = df.name.tolist()
c = {node1: {node2: round(haversine.haversine(
            (df.loc[df['name'] == node1, 'latitude'].iloc[0], df.loc[df['name'] == node1, 'longitude'].iloc[0]),
            (df.loc[df['name'] == node2, 'latitude'].iloc[0], df.loc[df['name'] == node2, 'longitude'].iloc[0])), 2)
            for node2 in nodes} 
     for node1 in nodes}

print(nodes)
print(c)

['Bahnhof', 'Friedrichsort', 'Holtenau', 'University', 'Mitte', 'Garden', 'Wellingdorf', 'Heikendorf', 'Laboe']
{'Bahnhof': {'Bahnhof': 0.0, 'Friedrichsort': 9.32, 'Holtenau': 6.68, 'University': 3.75, 'Mitte': 1.05, 'Garden': 1.24, 'Wellingdorf': 3.38, 'Heikendorf': 8.82, 'Laboe': 12.35}, 'Friedrichsort': {'Bahnhof': 9.32, 'Friedrichsort': 0.0, 'Holtenau': 3.12, 'University': 6.64, 'Mitte': 8.28, 'Garden': 9.22, 'Wellingdorf': 7.28, 'Heikendorf': 2.38, 'Laboe': 3.62}, 'Holtenau': {'Bahnhof': 6.68, 'Friedrichsort': 3.12, 'Holtenau': 0.0, 'University': 3.57, 'Mitte': 5.63, 'Garden': 6.86, 'Wellingdorf': 5.55, 'Heikendorf': 4.15, 'Laboe': 6.7}, 'University': {'Bahnhof': 3.75, 'Friedrichsort': 6.64, 'Holtenau': 3.57, 'University': 0.0, 'Mitte': 2.88, 'Garden': 4.44, 'Wellingdorf': 4.57, 'Heikendorf': 7.04, 'Laboe': 10.12}, 'Mitte': {'Bahnhof': 1.05, 'Friedrichsort': 8.28, 'Holtenau': 5.63, 'University': 2.88, 'Mitte': 0.0, 'Garden': 1.58, 'Wellingdorf': 2.82, 'Heikendorf': 7.85, 'Laboe': 

### Create models with PuLP

#### Load packages

In [9]:
# load the required modules
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 72.7 MB/s eta 0:00:00


#### Formulation of Miller, Tucker and Zemlin (1960)

$
\begin{align}
  \begin{array}{lll}
    &\textbf{Objective} & \\
    & \min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij} &~~~ (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & \sum\limits_{i=1}^{N} x_{ij} = 1,~ \forall~ j = 1,\dots,N  &~~~ (2) \\
    & \sum\limits_{j=1}^{N} x_{ij} = 1,~ \forall~ i = 1,\dots,N  &~~~ (3) \\
    & x_{ii} = 0,~ \forall~ i = 1,\dots,N  &~~~ (4) \\
    & u_{i} - u_{j} + N \cdot x_{ij} \le N -1,~ \forall~ i,j = 1,\dots,N: j \ne 1 ~\text{and}~ i \ne j  &~~~ (5) \\
    & u_{i} \in \mathbb{Z}^{\text{nonneg}},~ \forall~ i = 1,\dots,N  &~~~ (6) \\
    & x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N  &~~~ (7) \\
  \end{array}
\end{align}
$

In [28]:
# build problem
model = pulp.LpProblem(name='TSP_MTZ', sense=pulp.constants.LpMinimize)


# decision variables
x = pulp.LpVariable.dicts(name='x', indices=(nodes, nodes), cat='Binary')
u = pulp.LpVariable.dicts(name='u', indices=nodes, 
                          lowBound=1, upBound=len(nodes), 
                          cat='Integer')

# Objective (1)
model += pulp.lpSum(c[i][j] * x[i][j] for i in nodes for j in nodes), "(1):Objective"

# Constraints (2)
for j in nodes:
  model += pulp.lpSum(x[i][j] for i in nodes) == 1, '(2):' + j

# Constraints (3)
for i in nodes:
  model += pulp.lpSum(x[i][j] for j in nodes) == 1, '(3):' + i

# Constraints (4)
for i in nodes:
  model += x[i][i] == 0, '(4): ' + i

# Constraints (5)
for i in nodes:
  for j in nodes[1:]:
    if i != j:
      model += u[i] - u[j] + len(nodes)*x[i][j] <= len(nodes) - 1, '(5):' + i + '>' + j

# print model
# print(model)

### Solve model

In [29]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

#print model variable
# for v in model.variables():
#     print(v.name,"=", v.varValue)

print('Tour:')
tour = [] 
i = nodes[0]
tour_completed = False
while not tour_completed:
  for j in nodes:
    if x[i][j].varValue > 0: 
      print(' ' + str(x[i][j]) + ': ' + str(x[i][j].varValue) + ', c_ij=' + str(c[i][j]))
      tour.append(i)
      i = j
      if j == nodes[0]:
        tour_completed = True
      break
print(' ' + str(tour))

Status: Optimal
Objective value: 27.69
Tour:
 x_Bahnhof_Mitte: 1.0, c_ij=1.05
 x_Mitte_University: 1.0, c_ij=2.88
 x_University_Holtenau: 1.0, c_ij=3.57
 x_Holtenau_Friedrichsort: 1.0, c_ij=3.12
 x_Friedrichsort_Laboe: 1.0, c_ij=3.62
 x_Laboe_Heikendorf: 1.0, c_ij=3.6
 x_Heikendorf_Wellingdorf: 1.0, c_ij=6.1
 x_Wellingdorf_Garden: 1.0, c_ij=2.51
 x_Garden_Bahnhof: 1.0, c_ij=1.24
 ['Bahnhof', 'Mitte', 'University', 'Holtenau', 'Friedrichsort', 'Laboe', 'Heikendorf', 'Wellingdorf', 'Garden']


### Graphical visualization

In [30]:
# create new dataframe with the nodes sorted in the direction of the tour
current_node = nodes[0]
df_tour = pd.DataFrame(columns=df.columns)
df_tour = pd.concat([df_tour, df[df['name'] == current_node]])

while len(df_tour.index) < len(nodes) + 1:
  for j in nodes:
    if x[current_node][j].varValue > 0:      
      df_tour = pd.concat([df_tour, df[df['name'] == j]])
      current_node = j
      break;

print(df_tour)

fig = px.line_mapbox(df_tour, lat="latitude", lon="longitude", zoom=10, 
                    hover_name="name", width=800, height=800)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

            name   latitude  longitude
0        Bahnhof  54.315487  10.132285
4          Mitte  54.324606  10.136630
3     University  54.348125  10.117918
2       Holtenau  54.374804  10.148470
1  Friedrichsort  54.393713  10.184142
8          Laboe  54.409884  10.232679
7     Heikendorf  54.379798  10.212037
6    Wellingdorf  54.328293  10.179582
5         Garden  54.313102  10.150910
0        Bahnhof  54.315487  10.132285


## Assignments

#### Formulation of Gavish and Graves (1976)

$
\begin{align}
  \begin{array}{lll}
    &\textbf{Objective} & \\
    & \min \sum_{i=1,\dots,N} \sum_{j=1,\dots,N} c_{ij} \cdot x_{ij} &~~~ (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & \sum\limits_{i=1}^{N} x_{ij} = 1,~ \forall~ j = 1,\dots,N  &~~~ (2) \\
    & \sum\limits_{j=1}^{N} x_{ij} = 1,~ \forall~ i = 1,\dots,N  &~~~ (3) \\
    & x_{ii} = 0,~ \forall~ i = 1,\dots,N  &~~~ (4) \\
    &  \sum\limits_{j=1}^{N} z_{ij} - \sum\limits_{j=2}^{N} z_{ji} = 1,~ \forall~ i = 2,\dots,N &~~~ (5) \\
    & z_{ij} \le x_{ij} \cdot (N - 1),~ \forall~ i = 2,\dots,N, j = 1,\dots,N &~~~ (6) \\
    & z_{ij} \in \mathbb{Z}^{\text{nonneg}},~ \forall~ i,j = 1,\dots,N  &~~~ (7) \\
    & x_{ij} \in \{0,1\},~ \forall~ i,j = 1,\dots,N  &~~~ (8) \\
  \end{array}
\end{align}
$


In [ ]:
# initialize sets
nodes = df.name

# decision variables
x = pulp.LpVariable.dicts(name='x', indices=(nodes, nodes), cat='Binary')
z = pulp.LpVariable.dicts(name='z', lowBound=0, indices=(nodes, nodes), cat='Integer')

# build problem
model = pulp.LpProblem(name='TSP_GG', sense=pulp.constants.LpMinimize)

# Objective (1)
model += pulp.lpSum(c[i][j] * x[i][j] for i in nodes for j in nodes), "(1):Objective"

# Constraints (2)
for j in nodes:
  model += pulp.lpSum(x[i][j] for i in nodes) == 1, '(2):' + j

# Constraints (3)
for i in nodes:
  model += pulp.lpSum(x[i][j] for j in nodes) == 1, '(3):' + i

# Constraints (4)
for i in nodes:
  model += x[i][i] == 0, '(4): ' + i

# Constraints (5)
for i in nodes[1:]:
    model += pulp.lpSum(z[i][j] for j in nodes) - pulp.lpSum(z[j][i] for j in nodes[1:]) == 1, '(5):' + i

# Constraints (6)
for i in nodes[1:]:
    for j in nodes:
      model += z[i][j] <= x[i][j] * (len(nodes) - 1), '(6):' + i + '>' + j

# print model
# print(model)

### Computational experiments

In [34]:
# define MTZ formulation as function
def tsp_mtz(nodes, c):
  # build problem
  model = pulp.LpProblem(name='TSP_MTZ', sense=pulp.constants.LpMinimize)


  # decision variables
  x = pulp.LpVariable.dicts(name='x', indices=(nodes, nodes), cat='Binary')
  u = pulp.LpVariable.dicts(name='u', indices=nodes, 
                            lowBound=1, upBound=len(nodes), 
                            cat='Integer')

  # Objective (1)
  model += pulp.lpSum(c[i][j] * x[i][j] for i in nodes for j in nodes), "(1):Objective"

  # Constraints (2)
  for j in nodes:
    model += pulp.lpSum(x[i][j] for i in nodes) == 1, '(2):' + j

  # Constraints (3)
  for i in nodes:
    model += pulp.lpSum(x[i][j] for j in nodes) == 1, '(3):' + i

  # Constraints (4)
  for i in nodes:
    model += x[i][i] == 0, '(4): ' + i

  # Constraints (5)
  for i in nodes:
    for j in nodes[1:]:
      if i != j:
        model += u[i] - u[j] + len(nodes)*x[i][j] <= len(nodes) - 1, '(5):' + i + '>' + j
  
  return model

# define function to solve a model and returning the objective value
def solve_tsp(model):
  # solve problem
  model.solve()

  # get status
  print("Status:", pulp.LpStatus[model.status])

  # get objective value
  print('Objective value:', round(pulp.value(model.objective), 2))

  return(round(pulp.value(model.objective), 2))



#### Experiment 1: Weight in the cost function

In [42]:
x_values = []
y_values = []

for p in [1, 2, 4, 5, 10]:
  nodes = df.name.tolist()
  c = {node1: {node2: p * round(haversine.haversine(
              (df.loc[df['name'] == node1, 'latitude'].iloc[0], df.loc[df['name'] == node1, 'longitude'].iloc[0]),
              (df.loc[df['name'] == node2, 'latitude'].iloc[0], df.loc[df['name'] == node2, 'longitude'].iloc[0])), 2)
              for node2 in nodes} 
      for node1 in nodes}

  model = tsp_mtz(nodes=nodes, c=c)
  obj_value = solve_tsp(model=model)

  x_values.append(p)
  y_values.append(obj_value)

fig = px.line(x=x_values, y=y_values, markers=True)
fig.show()

Status: Optimal
Objective value: 27.69
Status: Optimal
Objective value: 55.38
Status: Optimal
Objective value: 110.76
Status: Optimal
Objective value: 138.45
Status: Optimal
Objective value: 276.9


#### Experiment 2: Distance calculation

In [47]:
x_values = []
y_values = []

for p in ['haversine', 'manhattan']:
  nodes = df.name.tolist()

  if p == 'haversine':
    c = {node1: {node2: round(haversine.haversine(
                (df.loc[df['name'] == node1, 'latitude'].iloc[0], df.loc[df['name'] == node1, 'longitude'].iloc[0]),
                (df.loc[df['name'] == node2, 'latitude'].iloc[0], df.loc[df['name'] == node2, 'longitude'].iloc[0])), 2)
                for node2 in nodes} 
        for node1 in nodes}
  elif p == 'manhattan':
    c = {node1: {node2: round(
        abs(df.loc[df['name'] == node1, 'latitude'].iloc[0] - df.loc[df['name'] == node2, 'latitude'].iloc[0]) + 
        abs(df.loc[df['name'] == node1, 'longitude'].iloc[0] - df.loc[df['name'] == node2, 'longitude'].iloc[0])        
        , 2) for node2 in nodes} 
         for node1 in nodes}

  model = tsp_mtz(nodes=nodes, c=c)
  obj_value = solve_tsp(model=model)

  x_values.append(p)
  y_values.append(obj_value)

fig = px.line(x=x_values, y=y_values, markers=True)
fig.show()

Status: Optimal
Objective value: 27.69
Status: Optimal
Objective value: 0.41
